In [1]:
import numpy as np

import os
from os.path import  join

In [2]:
#clean_data_path = "D://test_neurolab//train//clean"
clean_data_path = "C://input//neurolab//train//train//clean"

In [3]:
st = 0 
clean_data = []
for folder in os.listdir(clean_data_path):
    for file in os.listdir(join(clean_data_path, folder)):
        data = np.load(join(clean_data_path, folder, file))
        clean_data.append(data)
        #clead_data_sizes.append(data.size)
        #clead_data_shape.append(data.shape)

In [4]:
#noisy_data_path = "D://test_neurolab//train//noisy"
noisy_data_path = "C://input//neurolab//train//train//noisy"

In [5]:
noisy_data = []
for folder in os.listdir(noisy_data_path):
    for file in os.listdir(join(noisy_data_path, folder)):
        data= np.load(join(noisy_data_path, folder, file))
        noisy_data.append(data)

In [6]:
ex_data = clean_data[1]

In [7]:
def get_rect_windows(mel, debug = False):
    if (debug): print("mel size", mel.shape)
    windows = []
    for i in range(mel.shape[0]//80 + 1):
        if (80*(i+1)-1 < mel.shape[0]):
            if (debug): print("current size", 80*i, 80*(i+1)-1)
                
            ixgrid = np.ix_(range(80*i, 80*(i+1)), range(80))
            sample = mel[ixgrid]
            sample = sample.flatten()
            windows.append(sample)
            
            if (debug): print(sample.shape)
        elif(80 - (80*(i+1)-1 - mel.shape[0]) > 30): 
            
            if (debug): print("current size", 80*i, 80*(i+1)-1)
            ixgrid = np.ix_(range(80*i, mel.shape[0]), range(80))
            if (debug): print(mel[ixgrid].shape)                
            pad_matrix = np.pad(mel[ixgrid], ((0, 80 - mel[ixgrid].shape[0]), (0, 0)), 'reflect')
            
            pad_matrix = pad_matrix.flatten()  
            if (debug): print(pad_matrix.shape) 
            windows.append(pad_matrix)
        else:
            if (debug): print("not enough real data left", 80 - (80*(i+1)-1 - mel.shape[0]))
    return windows
a = get_rect_windows(ex_data, debug = True)

mel size (658, 80)
current size 0 79
(6400,)
current size 80 159
(6400,)
current size 160 239
(6400,)
current size 240 319
(6400,)
current size 320 399
(6400,)
current size 400 479
(6400,)
current size 480 559
(6400,)
current size 560 639
(6400,)
not enough real data left 19


In [8]:
clean_data_wind = []
for mel in clean_data:
    new_windows = get_rect_windows(mel)
    clean_data_wind.extend(new_windows)

In [9]:
noisy_data_wind = []
for mel in noisy_data:
    new_windows = get_rect_windows(mel)
    noisy_data_wind.extend(new_windows)

In [27]:
len(clean_data_wind),len(noisy_data_wind)

(115403, 115403)

In [33]:
a = [1,2] 
b = [2,3]
c = a.extend(b)
a,c

([1, 2, 2, 3], None)

In [10]:
y_train_clean = [0 for _ in range(len(clean_data_wind))]
y_train_noisy = [1 for _ in range(len(noisy_data_wind))]
y_train = y_train_clean + y_train_noisy

In [39]:
len(y_train)

230806

In [11]:
x_train = clean_data_wind + noisy_data_wind

In [43]:
len(X)

230806

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import pickle

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV

In [43]:
len(c),len(y_train)

(230806, 230806)

In [ ]:
logreg = LogisticRegression(class_weight='balanced', verbose = 15)
param = {'C':[0.001,0.003,0.005,0.01,0.03,0.05,0.1,0.3,0.5,1,2,3,3,4,5,10,20]}
clf = GridSearchCV(logreg,param,scoring='roc_auc',refit=True,cv=3)
clf.fit(X,y)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

In [14]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [36]:
filename = 'logreg.sav'
pickle.dump(logreg, open(filename, 'wb'))

In [20]:
logreg = pickle.load(open('logreg.sav', 'rb'))

In [23]:
ex_data = clean_data[1]
ex_data.shape

(658, 80)

In [26]:
logreg.predict(noisy_data_wind[0].reshape(1, -1))

array([1])

In [24]:
x_train[0]

array([ 0.64111328,  0.61132812,  0.54296875, ...,  0.03414917,
        0.05047607,  0.14135742], dtype=float16)

In [28]:

def predict_by_mel(mel, model,debug = False):
    if (debug): print("mel size", mel.shape)
    predictions = []
    for i in range(mel.shape[0]//80 + 1):
        if (80*(i+1)-1 < mel.shape[0]):
            if (debug): print("current size", 80*i, 80*(i+1)-1)
                
            ixgrid = np.ix_(range(80*i, 80*(i+1)), range(80))
            sample = mel[ixgrid]
            sample = sample.flatten()
            
            current_prediction = model.predict(sample.reshape(1, -1))
            
            predictions.append(current_prediction)
            
            #if (debug): print(sample.shape)
            if (debug): print(current_prediction)
        elif(80 - (80*(i+1)-1 - mel.shape[0]) > 30): 
            
            if (debug): print("current size", 80*i, 80*(i+1)-1)
            ixgrid = np.ix_(range(80*i, mel.shape[0]), range(80))
            if (debug): print(mel[ixgrid].shape)                
            pad_matrix = np.pad(mel[ixgrid], ((0, 80 - mel[ixgrid].shape[0]), (0, 0)), 'reflect')
            pad_matrix = pad_matrix.flatten()  
            if (debug): print(pad_matrix.shape) 
            current_prediction = model.predict(pad_matrix.reshape(1, -1))   
            predictions.append(current_prediction)
        else:
            if (debug): print("not enough real data left", 80 - (80*(i+1)-1 - mel.shape[0]))
    av_predictions = np.mean(predictions) 
    if (debug) : print(predictions)   
    if(av_predictions > 0.5):
        final_prediction = 1
    else:
        final_prediction = 0
         
    return final_prediction
ex_data = clean_data[1]
predict_by_mel(ex_data, logreg, debug = True)

mel size (658, 80)
current size 0 79
[0]
current size 80 159
[0]
current size 160 239
[0]
current size 240 319
[0]
current size 320 399
[0]
current size 400 479
[0]
current size 480 559
[0]
current size 560 639
[0]
not enough real data left 19
[array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]


0

# VALIDATION

In [12]:
clean_data_path_val = "C://input//neurolab//val//clean"
clean_data_val = []
for folder in os.listdir(clean_data_path_val):
    for file in os.listdir(join(clean_data_path_val, folder)):
        data = np.load(join(clean_data_path_val, folder, file))
        clean_data_val.append(data)
        
noisy_data_path_val = "C://input//neurolab//val//noisy"
noisy_data_val = []
for folder in os.listdir(noisy_data_path_val):
    for file in os.listdir(join(noisy_data_path_val, folder)):
        data= np.load(join(noisy_data_path_val, folder, file))
        noisy_data_val.append(data)



In [13]:
x_test = clean_data_val + noisy_data_val
y_train_clean_val = [0 for _ in range(len(clean_data_val))]
y_train_noisy_val = [1 for _ in range(len(noisy_data_val))]
y_test = y_train_clean_val + y_train_noisy_val

In [18]:
import tqdm

In [37]:
X_val[0].shape

(685, 80)

In [26]:
y_predict = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], logreg)
    y_predict.append(pred)
    

In [28]:
confusion_matrix = confusion_matrix(y_test, y_predict)
print(confusion_matrix)

[[1825  175]
 [ 220 1780]]


In [32]:
print (classification_report(y_val, y_predict))

             precision    recall  f1-score   support

          0       0.89      0.91      0.90      2000
          1       0.91      0.89      0.90      2000

avg / total       0.90      0.90      0.90      4000



In [ ]:
https://lukesingham.com/whos-going-to-leave-next/

In [31]:
from sklearn import metrics

In [33]:
from sklearn.tree import DecisionTreeClassifier

In [34]:
tree_model = DecisionTreeClassifier(max_depth=3)  
# Fit a decision tree
tree_model = tree_model.fit(x_train, y_train)  
# Training accuracy
tree_model.score(x_train, y_train)

# Predictions/probs on the test dataset
predicted = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], tree_model)
    predicted.append(pred)

# Store metrics
tree_accuracy = metrics.accuracy_score(y_test, predicted)  
#tree_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
tree_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
tree_classification_report = metrics.classification_report(y_test, predicted)  
tree_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
tree_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
tree_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# evaluate the model using 10-fold cross-validation
#tree_cv_scores = cross_val_score(tree.DecisionTreeClassifier(max_depth=3), x_test, y_test, scoring='precision', cv=10)

In [35]:
tree_model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [36]:
filename = 'tree_model.sav'
pickle.dump(tree_model, open(filename, 'wb'))

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
rf = RandomForestClassifier()  
# Fit
rf_model = rf.fit(x_train, y_train)  
rf_model.score(x_train, y_train)

0.99419859102449681

In [32]:
filename = 'rf.sav'
pickle.dump(rf, open(filename, 'wb'))

In [37]:


# Predictions/probs on the test dataset
#predicted = pd.DataFrame(rf_model.predict(x_test))  
predicted = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], rf_model)
    predicted.append(pred)
    
#probs = pd.DataFrame(rf_model.predict_proba(x_test))

# Store metrics
rf_accuracy = metrics.accuracy_score(y_test, predicted)  
#rf_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
rf_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
rf_classification_report = metrics.classification_report(y_test, predicted)  
rf_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
rf_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
rf_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# Evaluate the model using 10-fold cross-validation
#rf_cv_scores = cross_val_score(RandomForestClassifier(), x_test, y_test, scoring='precision', cv=10)  
#rf_cv_mean = np.mean(rf_cv_scores)

In [38]:
from sklearn.svm import SVC

In [ ]:
# Instantiate
svm_model = SVC(probability=True)  
# Fit
svm_model = svm_model.fit(x_train, y_train)  
# Accuracy
svm_model.score(x_train, y_train)

# Predictions/probs on the test dataset
predicted = pd.DataFrame(svm_model.predict(x_test))  
#probs = pd.DataFrame(svm_model.predict_proba(x_test))
predicted = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], svm_model)
    predicted.append(pred)

# Store metrics
svm_accuracy = metrics.accuracy_score(y_test, predicted)  
#svm_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
svm_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
svm_classification_report = metrics.classification_report(y_test, predicted)  
svm_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
svm_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
svm_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# Evaluate the model using 10-fold cross-validation
#svm_cv_scores = cross_val_score(SVC(probability=True), x_test, y_test, scoring='precision', cv=10)  
#svm_cv_mean = np.mean(svm_cv_scores) 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# instantiate learning model (k = 3)
knn_model = KNeighborsClassifier(n_neighbors=3)  
# fit the model
knn_model.fit(x_train, y_train)  
# Accuracy
knn_model.score(x_train, y_train)

# Predictions/probs on the test dataset
predicted = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], knn_model)
    predicted.append(pred)

# Store metrics
knn_accuracy = metrics.accuracy_score(y_test, predicted)  
#knn_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
knn_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
knn_classification_report = metrics.classification_report(y_test, predicted)  
knn_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
knn_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
knn_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# Evaluate the model using 10-fold cross-validation
#knn_cv_scores = cross_val_score(KNeighborsClassifier(n_neighbors=3), x_test, y_test, scoring='precision', cv=10)  
#knn_cv_mean = np.mean(knn_cv_scores)

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Instantiate
bayes_model = GaussianNB()  
# Fit the model
bayes_model.fit(x_train, y_train)  
# Accuracy
bayes_model.score(x_train, y_train)

# Predictions/probs on the test dataset
predicted = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], bayes_model)
    predicted.append(pred)

# Store metrics
bayes_accuracy = metrics.accuracy_score(y_test, predicted)  
#bayes_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
bayes_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
bayes_classification_report = metrics.classification_report(y_test, predicted)  
bayes_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
bayes_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
bayes_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# Evaluate the model using 10-fold cross-validation
#bayes_cv_scores = cross_val_score(KNeighborsClassifier(n_neighbors=3), x_test, y_test, scoring='precision', cv=10)  
#bayes_cv_mean = np.mean(bayes_cv_scores) 

In [ ]:
#logreg = LogisticRegression()
#logreg.fit(x_train, y_train)

predicted = []
for ind in range(len(x_test)):
    pred = predict_by_mel(x_test[ind], logreg)
    predicted.append(pred)

logit_accuracy = metrics.accuracy_score(y_test, predicted)  
#svm_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
logit_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
logit_classification_report = metrics.classification_report(y_test, predicted)  
logit_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
logit_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
logit_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# Evaluate the model using 10-fold cross-validation
logit_cv_scores = cross_val_score(LogisticRegression(probability=True), x_test, y_test, scoring='precision', cv=10)  
logit_cv_mean = np.mean(logit_cv_scores)  

In [ ]:
import pandas as pd

In [ ]:
models = pd.DataFrame({  
  'Model': ['Logistic', 'd.Tree', 'r.f.', 'SVM', 'kNN',  'Bayes'],
  'Accuracy' : [logit_accuracy, tree_accuracy, rf_accuracy, svm_accuracy, knn_accuracy, bayes_accuracy],
  'Precision': [logit_precision, tree_precision, rf_precision, svm_precision, knn_precision, bayes_precision],
  'recall' : [logit_recall, tree_recall, rf_recall, svm_recall, knn_recall, bayes_recall],
  'F1' : [logit_f1, tree_f1, rf_f1, svm_f1, knn_f1, bayes_f1]
  #'cv_precision' : [logit_cv_mean, tree_cv_mean, rf_cv_mean, svm_cv_mean, knn_cv_mean, bayes_cv_mean]
})
# Print table and sort by test precision
models.sort_values(by='Precision', ascending=False)

Because the spectrograms have a large dynamic
range (eg. from 10−3
to more than 1010), we apply element-wise log-transform!!!!!

Finally, we apply zero-mean unit-variance normalization on the spectrograms.


In [ ]:
http://cs231n.stanford.edu/reports/2015/pdfs/Final_Report_mkayser_vzhong.pdf